# MVP phenotype terms, round 2, data cleaning 

# Init

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [2]:
import sys
from pathlib import Path

In [3]:
_pwd = Path(".").resolve()
print(_pwd)
sys.path.append(str(_pwd))

/data/ik18445_cache/projects/phenotype-mapping/analysis/notebooks/mvp_round_2


In [4]:
from typing import List
import re
import json

import pandas as pd
import janitor
import numpy as np
from pydash import py_

from common_funcs import utils
import mvp_funcs, mvp_types

In [5]:
proj_root = utils.find_project_root("docker-compose.yml")
assert proj_root.exists(), proj_root

data_path = proj_root / "data"
mvp_terms_path = data_path / "mvp-terms-2022-07" / "all_mapped_phenotypes_03JUN2022.csv"
mvp_terms_1_path = data_path / "mvp-terms-2022-09" / "mvp-terms-2022-09.csv"
assert mvp_terms_path.exists(), mvp_terms_path
assert mvp_terms_1_path.exists(), mvp_terms_1_path

output_path = data_path / "output" / "mvp-mapping-round-2"
output_path.mkdir(exist_ok=True, parents=False)

# raw datasets

In [6]:
df_raw = pd.read_csv(mvp_terms_path)
df_raw.info()
df_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2081 entries, 0 to 2080
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Phenotype      2081 non-null   object 
 1   description    2079 non-null   object 
 2   dataset        2081 non-null   object 
 3   MVP_Cases      2060 non-null   object 
 4   MVP_Controls   1977 non-null   float64
 5   trait_type     2081 non-null   object 
 6   UKBB_filename  1506 non-null   object 
 7   UKBB_Cases     1505 non-null   float64
 8   UKBB_Controls  1439 non-null   float64
 9   unmapped_plan  648 non-null    object 
dtypes: float64(3), object(7)
memory usage: 162.7+ KB


,Phenotype,description,dataset,MVP_Cases,MVP_Controls,trait_type,UKBB_filename,UKBB_Cases,UKBB_Controls,unmapped_plan
0,8,Intestinal infection,UKBB+MVP,9351,454638.0,phecode,phecode-008-both_sexes.tsv.bgz,13044.0,407487.0,NaN
1,8.5,Bacterial enteritis,UKBB+MVP,5609,458380.0,phecode,phecode-008.5-both_sexes.tsv.bgz,3275.0,407487.0,NaN
2,8.51,Intestinal e.coli,MVP,57,463932.0,Phecode,NaN,NaN,NaN,use MVP only
3,8.52,Intestinal infection due to C. difficile,UKBB+MVP,4873,459116.0,phecode,phecode-008.52-both_sexes.tsv.bgz,858.0,407487.0,NaN
4,8.6,Viral Enteritis,UKBB+MVP,1347,462642.0,phecode,phecode-008.6-both_sexes.tsv.bgz,1114.0,407487.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2076,DBP (at enrollment),Diastolic blood pressure,UKBB+MVP,"463,989",NaN,vital status,continuous-4079-both_sexes-irnt.tsv.bgz,396667.0,NaN,NaN
2077,P (Pulse at enrollment),Heart rate,UKBB+MVP,"463,989",NaN,vital status,continuous-102-both_sexes-irnt.tsv.bgz,396667.0,NaN,NaN
2078,Height (in),Heigth,MVP,"463,989",NaN,vital status,NaN,NaN,NaN,use MVP only
2079,Weight (lb),Weight,UKBB+MVP,"463,989",NaN,vital status,continuous-21002-both_sexes-irnt.tsv.bgz,419316.0,NaN,NaN


In [7]:
df_raw_1 = pd.read_csv(mvp_terms_1_path)
df_raw_1.info()
df_raw_1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3325 entries, 0 to 3324
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   phenocode  3325 non-null   object
 1   name       3325 non-null   object
 2   source     3325 non-null   object
dtypes: object(3)
memory usage: 78.1+ KB


,phenocode,name,source
0,AB1_AMOEBIASIS,Amoebiasis,FinnGen
1,AB1_ANOGENITAL_HERPES_SIMPLEX,Anogenital herpesviral [herpes simplex] infection,FinnGen
2,AB1_ARTHROPOD,Arthropod-borne viral fevers and viral haemorr...,FinnGen
3,AB1_ASPERGILLOSIS,Aspergillosis,FinnGen
4,AB1_ATYPICAL_CNS_VIRUS,Atypical virus infections of central nervous s...,FinnGen
...,...,...,...
3320,Spinal_canal_stenosis,Spinal canal stenosis,Biobank_Japan
3321,Mastopathy,Mastopathy,Biobank_Japan
3322,Schizophrenia,Schizophrenia,Biobank_Japan
3323,Tonsillitis,Tonsillitis,Biobank_Japan


# cleaning

## pre cleaning diagnostics

- check duplication in id, and id-label combination
- check id and label emptiness
- check category counts

In [8]:
df_raw[df_raw.duplicated(subset=["Phenotype", "description"], keep=False)]

,Phenotype,description,dataset,MVP_Cases,MVP_Controls,trait_type,UKBB_filename,UKBB_Cases,UKBB_Controls,unmapped_plan


In [9]:
df_raw[df_raw.duplicated(subset=["Phenotype"], keep=False)]

,Phenotype,description,dataset,MVP_Cases,MVP_Controls,trait_type,UKBB_filename,UKBB_Cases,UKBB_Controls,unmapped_plan
102,172.11,C43 Malignant melanoma of skin,UKBB+MVP,10578,453411.0,icd10,icd10-C43-both_sexes.tsv.bgz,2598.0,417933.0,NaN
103,172.11,D03 Melanoma in situ,UKBB+MVP,10578,453411.0,icd10,icd10-D03-both_sexes.tsv.bgz,956.0,419575.0,NaN
1290,620,N89 Other noninflammatory disorders of vagina,UKBB+MVP,492,463497.0,icd10,icd10-N89-both_sexes.tsv.bgz,1692.0,418839.0,NaN
1291,620,N90 Other noninflammatory disorders of vulva a...,UKBB+MVP,492,463497.0,icd10,icd10-N90-both_sexes.tsv.bgz,2175.0,418356.0,NaN
1859,1010.5,Z22 Carrier of infectious disease,UKBB+MVP,243866,220123.0,icd10,icd10-Z22-both_sexes.tsv.bgz,1472.0,419059.0,NaN
1860,1010.5,Z29 Need for other prophylactic measures,UKBB+MVP,243866,220123.0,icd10,icd10-Z29-both_sexes.tsv.bgz,888.0,419643.0,NaN
1861,1010.6,Z31 Procreative management,UKBB+MVP,5167,458822.0,icd10,icd10-Z31-both_sexes.tsv.bgz,322.0,420209.0,NaN
1862,1010.6,Z32 Pregnancy examination and test,UKBB+MVP,5167,458822.0,icd10,icd10-Z32-both_sexes.tsv.bgz,185.0,420346.0,NaN
1863,1010.6,Z36 Antenatal screening,UKBB+MVP,5167,458822.0,icd10,icd10-Z36-both_sexes.tsv.bgz,1293.0,419238.0,NaN
1864,1010.7,Z56 Problems related to employment and unemplo...,UKBB+MVP,380882,83107.0,icd10,icd10-Z56-both_sexes.tsv.bgz,182.0,420349.0,NaN


In [10]:
df_raw_1[df_raw_1.duplicated(subset=["phenocode", "name"], keep=False)]

,phenocode,name,source
1581,ILD,Interstitial lung disease,FinnGen
3202,ILD,Interstitial lung disease,Biobank_Japan


In [11]:
df_raw_1[df_raw_1.duplicated(subset=["phenocode"], keep=False)]

,phenocode,name,source
1581,ILD,Interstitial lung disease,FinnGen
2945,T1D,"Type1 diabetes, definitions combined",FinnGen
2952,T2D,"Type 2 diabetes, definitions combined",FinnGen
3111,T2D,Type 2 diabetes,Biobank_Japan
3202,ILD,Interstitial lung disease,Biobank_Japan
3209,T1D,Type 1 diabetes,Biobank_Japan


In [12]:
_cols = ["Phenotype", "description", "dataset"]
_df = df_raw
for _col in _cols:
    print(_col)
    _df_slice = _df[(_df[_col].isnull()) | (_df[_col] == "")]
    print(len(_df_slice))

_cols = ["phenocode", "name", "source"]
_df = df_raw_1
for _col in _cols:
    print(_col)
    _df_slice = _df[(_df[_col].isnull()) | (_df[_col] == "")]
    print(len(_df_slice))

Phenotype
0
description
2
dataset
0
phenocode
0
name
0
source
0


In [13]:
df_raw[df_raw["description"].isnull()]

,Phenotype,description,dataset,MVP_Cases,MVP_Controls,trait_type,UKBB_filename,UKBB_Cases,UKBB_Controls,unmapped_plan
1942,TotalCK,NaN,MVP,230038,NaN,lab,NaN,NaN,NaN,use MVP only
1945,Troponin,NaN,MVP,17905,NaN,lab,NaN,NaN,NaN,use MVP only


In [14]:
print(df_raw["dataset"].value_counts())
print(df_raw_1["source"].value_counts())

UKBB+MVP    1490
MVP          577
UKBB          14
Name: dataset, dtype: int64
FinnGen          3095
Biobank_Japan     230
Name: source, dtype: int64


## cleaning

- drop empty items
- create trait id
- reshape df into desired format

In [15]:
df_clean = (
    df_raw.also(lambda df: print("Init", len(df)))[
        ["Phenotype", "description", "dataset"]
    ]
    # drop dupes
    .drop_duplicates()
    .also(lambda df: print("Drop dupes", len(df)))
    # remove empty fields
    .dropna()
    .also(lambda df: print("Drop NA", len(df)))
    # trait id
    .reset_index(drop=True)
    .reset_index(drop=False)
    .assign(
        trait_id=lambda df: df.apply(
            lambda row: "{dataset}-{phenotype}-{index:02d}".format(
                dataset=row["dataset"],
                phenotype=mvp_funcs.clean_phenotype_id(row["Phenotype"]),
                index=row["index"],
            ),
            axis=1,
        )
    )
    .also(lambda df: print("IDs: {ids}".format(ids=df["trait_id"].tolist()[:5])))
    # trait term
    .assign(trait_term=lambda df: df["description"])
    # wrap up
    .assign(
        trait_basic_info=lambda df: df.apply(
            lambda row: {
                "Phenotype": row["Phenotype"],
                "description": row["description"],
                "dataset": row["dataset"],
            },
            axis=1,
        )
    )
    .drop(columns=["index", "Phenotype", "description", "dataset"])
)
df_clean.info()

Init 2081
Drop dupes 2081
Drop NA 2079
IDs: ['UKBB+MVP-8-00', 'UKBB+MVP-85-01', 'MVP-851-02', 'UKBB+MVP-852-03', 'UKBB+MVP-86-04']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2079 entries, 0 to 2078
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trait_id          2079 non-null   object
 1   trait_term        2079 non-null   object
 2   trait_basic_info  2079 non-null   object
dtypes: object(3)
memory usage: 48.9+ KB


In [16]:
df_clean_1 = (
    df_raw_1.also(lambda df: print("Init", len(df)))
    # drop dupes
    .drop_duplicates()
    .also(lambda df: print("Drop dupes", len(df)))
    # remove empty fields
    .dropna()
    .also(lambda df: print("Drop NA", len(df)))
    # trait id
    .reset_index(drop=True)
    .reset_index(drop=False)
    .assign(
        trait_id=lambda df: df.apply(
            lambda row: "{source}-{phenocode}-{index:02d}".format(
                source=row["source"],
                phenocode=row["phenocode"],
                index=row["index"],
            ),
            axis=1,
        )
    )
    .also(lambda df: print("IDs: {ids}".format(ids=df["trait_id"].tolist()[:5])))
    # trait term
    .assign(trait_term=lambda df: df["name"])
    # wrap up
    .assign(
        trait_basic_info=lambda df: df.apply(
            lambda row: {
                "phenocode": row["phenocode"],
                "name": row["name"],
                "source": row["source"],
            },
            axis=1,
        )
    )
    .drop(columns=["index", "phenocode", "name", "source"])
)
df_clean_1.info()

Init 3325
Drop dupes 3325
Drop NA 3325
IDs: ['FinnGen-AB1_AMOEBIASIS-00', 'FinnGen-AB1_ANOGENITAL_HERPES_SIMPLEX-01', 'FinnGen-AB1_ARTHROPOD-02', 'FinnGen-AB1_ASPERGILLOSIS-03', 'FinnGen-AB1_ATYPICAL_CNS_VIRUS-04']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3325 entries, 0 to 3324
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trait_id          3325 non-null   object
 1   trait_term        3325 non-null   object
 2   trait_basic_info  3325 non-null   object
dtypes: object(3)
memory usage: 78.1+ KB


In [17]:
COLS = ["trait_id", "trait_term", "trait_term_clean", "trait_basic_info"]

df_clean_merge = pd.concat([df_clean, df_clean_1]).reset_index(drop=True)
df_clean_merge = df_clean_merge.assign(
    trait_term_clean=lambda df: df["trait_term"].apply(mvp_funcs.clean_trait_term)
)[COLS]
mvp_types.CleanedDf.validate(df_clean_merge)
df_clean_merge.info()
df_clean_merge

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trait_id          5404 non-null   object
 1   trait_term        5404 non-null   object
 2   trait_term_clean  5404 non-null   object
 3   trait_basic_info  5404 non-null   object
dtypes: object(4)
memory usage: 169.0+ KB


,trait_id,trait_term,trait_term_clean,trait_basic_info
0,UKBB+MVP-8-00,Intestinal infection,Intestinal infection,"{'Phenotype': '8', 'description': 'Intestinal ..."
1,UKBB+MVP-85-01,Bacterial enteritis,Bacterial enteritis,"{'Phenotype': '8.5', 'description': 'Bacterial..."
2,MVP-851-02,Intestinal e.coli,Intestinal e.coli,"{'Phenotype': '8.51', 'description': 'Intestin..."
3,UKBB+MVP-852-03,Intestinal infection due to C. difficile,Intestinal infection due to C. difficile,"{'Phenotype': '8.52', 'description': 'Intestin..."
4,UKBB+MVP-86-04,Viral Enteritis,Viral Enteritis,"{'Phenotype': '8.6', 'description': 'Viral Ent..."
...,...,...,...,...
5399,Biobank_Japan-Spinal_canal_stenosis-3320,Spinal canal stenosis,Spinal canal stenosis,"{'phenocode': 'Spinal_canal_stenosis', 'name':..."
5400,Biobank_Japan-Mastopathy-3321,Mastopathy,Mastopathy,"{'phenocode': 'Mastopathy', 'name': 'Mastopath..."
5401,Biobank_Japan-Schizophrenia-3322,Schizophrenia,Schizophrenia,"{'phenocode': 'Schizophrenia', 'name': 'Schizo..."
5402,Biobank_Japan-Tonsillitis-3323,Tonsillitis,Tonsillitis,"{'phenocode': 'Tonsillitis', 'name': 'Tonsilli..."


## Post cleaning diagnostics

None at the moment

# wrap up

In [18]:
output_file = output_path / "mvp-terms-clean.json"
with output_file.open("w") as f:
    json.dump(df_clean_merge.to_dict(orient="records"), f)

In [19]:
FLAT_COLS = ["trait_id", "trait_term", "trait_term_clean"]

output_file_flat = output_path / "mvp-terms-clean-flat.csv"
df_clean_merge[FLAT_COLS].to_csv(output_file_flat, index=False)